In [4]:
import numpy as np
import random
import langchain
from langchain.llms.base import LLM
import json
import re
import sqlite3
from datetime import datetime
import pandas as np
from collections import Counter
import pandas as pd
from config import API_KEY

In [30]:
#分配角色
char_quota={"seer":1,"witch":1,"hunter":1,"villager":2,"werewolf":3}

In [44]:
#存活状态确认
aliv_list=list(range(8))
def char_alive_check(aliv_list,char_dist):
    char_aliv=[]
    for i in aliv_list:
        char_aliv.append(char_dist[i])
    return set(char_aliv)
a=char_alive_check(aliv_list,char_dist)

In [28]:
#角色初始定义

In [5]:
#大模型agent定义
import random
from http import HTTPStatus
import dashscope
from dashscope import Generation  # 建议dashscope SDK 的版本 >= 1.14.0

class base_llm():
    def __init__(self,system_prompt):
        self.api_key=API_KEY
        self.system_prompt=system_prompt
    def call_with_messages(self,prompt_info):
        messages = [{'role': 'system', 'content': self.system_prompt},
                    {'role': 'user', 'content': prompt_info}]
        dashscope.api_key=self.api_key
        response = Generation.call(model="qwen-plus",
                                   messages=messages,
                                   # 设置随机数种子seed，如果没有设置，则随机数种子默认为1234
                                   seed=random.randint(1, 10000),
                                   # 将输出设置为"message"格式
                                   result_format='message')
        if response.status_code == HTTPStatus.OK:
            return response.output.choices[0].message.content
        else:
            return ('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
                response.request_id, response.status_code,
                response.code, response.message))

In [152]:
a=base_llm(system_prompt="你是一个聪明的助理")
b=a.call_with_messages("世界上最高的山峰是什么")
b

'珠穆朗玛峰（Mount Everest）是世界上海拔最高的山峰，位于喜马拉雅山脉中，其海拔高度为8,848.86米（29,031.7英尺），这是2020年中国和尼泊尔共同宣布的最新测量数据。'

In [150]:
# b.output.choices[0].message.content
# def extract_content(response):
#     response=json.loads(response)
#     for choice in response.get('output', {}).get('choices', []):
#         if 'message' in choice and 'content' in choice['message']:
#             return choice['message']['content']
#     return None

# # 使用函数提取content
# content = extract_content(b)
# print(content)

'珠穆朗玛峰，位于喜马拉雅山脉中，是世界海拔最高的山峰，其海拔高度为8,848.86米（2020年最新测量数据）。'

In [102]:
game_intro='''《狼人杀》是一款社交推理游戏，游戏模拟了一个小村庄里村民与狼人之间的斗争。游戏目标根据不同角色有所不同，村民方需要找出并消灭所有狼人，狼人方则要尽可能消灭村民，直至狼人数量等于或多于村民数量。
根据你提供的角色设置，这里有1个预言家、1个女巫、1个猎人、2个村民和3个狼人。下面是这些角色的基本职能和游戏的简要规则：
- 预言家：每个夜晚可以查验一名玩家的身份，了解这名玩家是村民方还是狼人方。
- 女巫：拥有两瓶药水，一瓶可以救活当夜被狼人杀害的玩家，另一瓶可以在夜晚毒杀一名玩家。每瓶药水只能使用一次。
- 猎人：在被淘汰时（不论是白天被投票淘汰还是夜里被狼人杀害），可以开枪带走一名玩家。
- 村民：没有特殊能力，主要通过讨论和投票来识别狼人。
- 狼人：每个夜晚会选择杀害一名玩家，目标是消灭所有村民，获得游戏胜利。
游戏流程大致如下：
- 游戏开始时，每位玩家随机分配一个角色，并保密自己的身份。
- 游戏分为夜晚和白天两个阶段。
- 夜晚：所有玩家闭眼，游戏主持人（通常称为法官）指挥特定角色的玩家按顺序睁眼执行他们的能力。首先是狼人相互确认身份，然后选择一个玩家作为杀害目标。随后，预言家可以查验一名玩家的身份，女巫会被告知狼人的杀害目标，决定是否使用药水。
- 白天：所有玩家睁眼。如果有玩家在夜里被杀害，法官会宣布该玩家死亡（除非被女巫救活）。然后所有存活的玩家进行讨论，最后进行投票，票数最多的玩家被淘汰出局。如果猎人被淘汰，他可以使用他的技能。
- 游戏继续进行，直到村民方或狼人方达成胜利条件。村民方胜利条件是所有狼人被消灭，狼人方胜利条件是狼人数量等于或多于存活的非狼人玩家数量。
'''
char_intro_werewolf='''你在玩一局狼人杀游戏，这是一个8人局，其中包含3个狼人、2个村民和3个神（预言家、女巫和猎人），这局中你抽到的角色是狼人，游戏开始时你知道你的狼人队友身份，你不知道其他玩家的身份。
    你和你的狼人队友属于狼人阵营，你们的胜利条件是消灭所有的村民或者所有的神。你的特殊能力是和狼人队友在晚上讨论后消灭一名玩家。
    因此在游戏中你的工作包括以下内容：
    1.在夜晚和狼队友讨论后投票杀掉一个玩家。
    2.白天发表言论，保护自己和狼人队友，让大家投票杀掉一个玩家。
'''
char_intro_seer='''你在玩一局狼人杀游戏，这是一个8人局，其中包含3个狼人、2个村民和3个神（预言家、女巫和猎人），这局中你抽到的角色是预言家，游戏开始时你不知道其他玩家的身份。
    你和所有神以及村民属于人类阵营，你们的胜利条件是消灭所有的狼人。你的特殊能力是可以在夜晚验明一个玩家的真实身份。
    因此在游戏中你的工作包括以下内容：
    1.通过你的预言家身份，在夜晚查验某个玩家的身份。
    2.在白天，把你得到的信息告诉大家，如果你查验身份的玩家属于人类阵营，通过发言保护这个玩家不被投票消灭；如果你查验身份的玩家是狼人，引导大家用可能的方式消灭他。
'''
char_intro_witch='''你在玩一局狼人杀游戏，这是一个8人局，其中包含3个狼人、2个村民和3个神（预言家、女巫和猎人），这局中你抽到的角色是女巫，游戏开始时你不知道其他玩家的身份。
    你和所有神以及村民属于人类阵营，你们的胜利条件是消灭所有的狼人。你的特殊能力是有一瓶起死回生的解药和一瓶杀人的毒药，你可以在夜晚决定是否用解药救下被狼人杀掉的玩家，或者用毒药杀掉一个玩家，注意一局游戏中两种药都只能用一次。
    因此在游戏中你的工作包括以下内容：
    1.在夜晚，根据你对当前形式的判断，决定是否用解药救下某个被杀掉的玩家，或者是否用毒药杀掉某个玩家。
    2.在白天发表言论，根据你观察到的信息决定保护人类阵营的玩家，同时帮大家找到狼人并投票杀掉狼人玩家。
'''
char_intro_hunter='''你在玩一局狼人杀游戏，这是一个8人局，其中包含3个狼人、2个村民和3个神（预言家、女巫和猎人），这局中你抽到的角色是猎人，游戏开始时你不知道其他玩家的身份。
    你和所有神以及村民属于人类阵营，你们的胜利条件是消灭所有的狼人。你的特殊能力是有一把猎枪，当你以任意方式死亡时，你可以立即决定是否用猎枪消灭一名玩家。
    因此在游戏中你的工作包括以下内容：
    1.在白天发表言论，根据你的推理保护人类阵营玩家，找到狼人玩家并投票消灭他们。
    2.当你死亡时，根据你的判断决定是否杀掉某个你认为一定是狼人的玩家，或者不开枪以免误伤人类阵营队友；
'''
char_intro_villager='''你在玩一局狼人杀游戏，这是一个8人局，其中包含3个狼人、2个村民和3个神（预言家、女巫和猎人），这局中你抽到的角色是村民，游戏开始时你不知道其他玩家的身份。
    你和所有神以及村民属于人类阵营，你们的胜利条件是消灭所有的狼人。你没有任何特殊能力。
    因此在游戏中你的工作包括以下内容：
    1.在白天发表言论，根据你的推理保护人类阵营玩家，找到狼人玩家并投票消灭他们。
'''

In [205]:
def char_info_initialize(char_dist):
    char_info_dic={}
    for i in char_dist.keys():
        if char_dist[i]=='villager':
            info=game_intro+char_intro_villager+"你是%d号玩家，你的身份是村民。\n"%(i)
            char_info_dic[i]=info
        elif char_dist[i]=='seer':
            info=game_intro+char_intro_seer+"你是%d号玩家，你的身份是预言家。\n"%(i)
            char_info_dic[i]=info
        elif char_dist[i]=='witch':
            info=game_intro+char_intro_witch+"你是%d号玩家，你的身份是女巫。\n"%(i)
            char_info_dic[i]=info
        elif char_dist[i]=='hunter':
            info=game_intro+char_intro_hunter+"你是%d号玩家，你的身份是猎人。\n"%(i)
            char_info_dic[i]=info
        elif char_dist[i]=='werewolf':
            werewolf_list=[key for key,value in char_dist.items() if value=='werewolf']
            werewolf_list.remove(i)
            info=game_intro+char_intro_villager+"你是%d号玩家，你的身份是狼人，你的狼人队友包括"%(i)
            for j in werewolf_list:
                info+="%d号玩家、"%(j)
            info=info[:-1]+"。\n"
            char_info_dic[i]=info
    return char_info_dic

In [121]:
a=char_info_initialize(char_dist=char_dist)
a

{6: '《狼人杀》是一款社交推理游戏，游戏模拟了一个小村庄里村民与狼人之间的斗争。游戏目标根据不同角色有所不同，村民方需要找出并消灭所有狼人，狼人方则要尽可能消灭村民，直至狼人数量等于或多于村民数量。\n根据你提供的角色设置，这里有1个预言家、1个女巫、1个猎人、2个村民和3个狼人。下面是这些角色的基本职能和游戏的简要规则：\n- 预言家：每个夜晚可以查验一名玩家的身份，了解这名玩家是村民方还是狼人方。\n- 女巫：拥有两瓶药水，一瓶可以救活当夜被狼人杀害的玩家，另一瓶可以在夜晚毒杀一名玩家。每瓶药水只能使用一次。\n- 猎人：在被淘汰时（不论是白天被投票淘汰还是夜里被狼人杀害），可以开枪带走一名玩家。\n- 村民：没有特殊能力，主要通过讨论和投票来识别狼人。\n- 狼人：每个夜晚会选择杀害一名玩家，目标是消灭所有村民，获得游戏胜利。\n游戏流程大致如下：\n- 游戏开始时，每位玩家随机分配一个角色，并保密自己的身份。\n- 游戏分为夜晚和白天两个阶段。\n- 夜晚：所有玩家闭眼，游戏主持人（通常称为法官）指挥特定角色的玩家按顺序睁眼执行他们的能力。首先是狼人相互确认身份，然后选择一个玩家作为杀害目标。随后，预言家可以查验一名玩家的身份，女巫会被告知狼人的杀害目标，决定是否使用药水。\n- 白天：所有玩家睁眼。如果有玩家在夜里被杀害，法官会宣布该玩家死亡（除非被女巫救活）。然后所有存活的玩家进行讨论，最后进行投票，票数最多的玩家被淘汰出局。如果猎人被淘汰，他可以使用他的技能。\n- 游戏继续进行，直到村民方或狼人方达成胜利条件。村民方胜利条件是所有狼人被消灭，狼人方胜利条件是狼人数量等于或多于存活的非狼人玩家数量。\n你在玩一局狼人杀游戏，这是一个8人局，其中包含3个狼人、2个村民和3个神（预言家、女巫和猎人），这局中你抽到的角色是村民，游戏开始时你不知道其他玩家的身份。\n    你和所有神以及村民属于人类阵营，你们的胜利条件是消灭所有的狼人。你没有任何特殊能力。\n    因此在游戏中你的工作包括以下内容：\n    1.在白天发表言论，根据你的推理保护人类阵营玩家，找到狼人玩家并投票消灭他们。\n你是6号玩家，你的身份是预言家。',
 7: '《狼人杀》是一款社交推理游戏，游戏模拟了一个小村庄里村民与狼人之间的斗争。游戏目标根据不同角色有所不同，村民方需要找

In [363]:
#狼人
class werewolf():
    def __init__(self,char_info,num,logger):
        self.char_info=char_info
        self.llm=base_llm(self.char_info)
        self.num=num
        self.speaker='werewolf'
        self.logger=logger
    def get_cur_state(self):
        return self.logger.get_cur_log(self.speaker)
    def kill_speech(self,strategy=''):     #杀人发言
        cur_state=self.get_cur_state()
        prompt=cur_state+strategy+'''现在是晚上时间，请你根据【游戏记录】中的信息，结合【游戏策略】，请仔细思考应该消灭哪一个玩家，并在和其他狼人的讨论中发言。
        请仔细思考，然后把发言内容写到【发言内容】中,发言内容请简洁，必须控制在150字内。
        【发言内容】:'''
        result=self.llm.call_with_messages(prompt)
#         res=re.search(r"【发言内容】(.*)",result).group(1)[1:]
        res=result
        self.logger.add_log('夜晚狼人杀人发言',self.num,self.speaker,res)
        return res
    def kill_vote(self,strategy=''):     #杀人投票
        cur_state=self.get_cur_state()
        prompt=cur_state+strategy+'''现在是晚上时间，请你根据【游戏记录】中的信息，结合【游戏策略】，作为一个狼人玩家，请仔细思考决定投票消灭哪一个玩家,并给出一个投票结果。
        请仔细思考，然后把投票结果写到【投票结果】中。
        【投票结果】:'''
        result=self.llm.call_with_messages(prompt)
#         res=re.search(r"【投票结果】(.*)",result).group(1)[1:]
        res=result
        self.logger.add_log('夜晚狼人杀人投票',self.num,self.speaker,res)
        return res
    def speech(self,strategy=''):    #白天发言
        cur_state=self.get_cur_state()
        prompt=cur_state+strategy+'''现在是白天时间轮到你发言，请你根据【游戏记录】中的信息，结合【游戏策略】，请仔细思考如何发言才能获胜，并发表你的看法，误导人类阵营。
        请仔细思考，然后把发言内容写到【发言内容】中。
        【发言内容】:'''
        result=self.llm.call_with_messages(prompt)
#         res=re.search(r"【发言内容】(.*)",result).group(1)[1:]
        res=result
        self.logger.add_log('白天发言',self.num,self.speaker,res)
        return res
    def day_vote(self,strategy=''):    #白天投票
        cur_state=self.get_cur_state()
        prompt=cur_state+strategy+'''现在是白天投票时间，请你根据【游戏记录】中今天的对话和历史信息，结合【游戏策略】，仔细思考投票决定消灭哪个玩家。
        请仔细思考，然后把投票结果写到【投票结果】中。
        【投票结果】:'''
        result=self.llm.call_with_messages(prompt)
#         res=re.search(r"【投票结果】(.*)",result).group(1)[1:]
        res=result
        self.logger.add_log('白天投票',self.num,self.speaker,res)
        return res

In [333]:
#seer
class seer():
    def __init__(self,char_info,num,logger):
        self.char_info=char_info
        self.llm=base_llm(self.char_info)
        self.num=num
        self.speaker='seer'
        self.logger=logger
    def get_cur_state(self):
        return self.logger.get_cur_log(self.speaker)
    def char_val(self,strategy=''):     #验身份
        cur_state=self.get_cur_state()
        prompt=cur_state+strategy+'''现在是晚上时间，请你根据【游戏记录】中的信息，结合【游戏策略】，请仔细思考验证哪一个嫌疑最大的玩家。
        请仔细思考中，然后把验身份的对象玩家编号写到【玩家编号】中。
        【玩家编号】:'''
        result=self.llm.call_with_messages(prompt)
#         res=re.search(r"【玩家编号】(.*)",result).group(1)[1:]
        res=result
        self.logger.add_log('预言家验证身份',self.num,self.speaker,res)
        return res
    def speech(self,strategy=''):    #白天发言
        cur_state=self.get_cur_state()
        prompt=cur_state+strategy+'''现在是白天时间轮到你发言，请你根据【游戏记录】中的信息，结合【游戏策略】和你获得的信息，仔细思考如何发言才能帮助人类阵营获胜，并发表你的看法。
        请仔细思考，然后把发言内容写到【发言内容】中。
        【发言内容】:'''
        result=self.llm.call_with_messages(prompt)
#         res=re.search(r"【发言内容】(.*)",result).group(1)[1:]
        res=result
        self.logger.add_log('白天发言',self.num,self.speaker,res)
        return res
    def day_vote(self,strategy=''):    #白天投票
        cur_state=self.get_cur_state()
        prompt=cur_state+strategy+'''现在是白天投票时间，请你根据【游戏记录】中今天的对话和历史信息，结合【游戏策略】，仔细思考投票决定消灭哪个玩家。
        请仔细思考，然后把投票结果写到【投票结果】中。
        【投票结果】:'''
        result=self.llm.call_with_messages(prompt)
#         res=re.search(r"【投票结果】(.*)",result).group(1)[1:]
        res=result
        self.logger.add_log('白天投票',self.num,self.speaker,res)
        return res

In [315]:
#witch
class witch():
    def __init__(self,char_info,num,logger):
        self.char_info=char_info
        self.llm=base_llm(self.char_info)
        self.num=num
        self.speaker='witch'
        self.logger=logger
        self.poison=1
        self.antidote=1
    def get_cur_state(self):
        return self.logger.get_cur_log(self.speaker)
    def use_poison(self,strategy=''):     #毒药
        cur_state=self.get_cur_state()
        prompt=cur_state+strategy+'''现在是晚上时间，请你根据【游戏记录】中的信息，结合【游戏策略】，请仔细思考是否使用女巫的毒药消灭一名玩家，以及消灭哪位玩家。
        请仔细思考，然后把你的决定写到【毒药决策】中。
        【毒药决策】:'''
        result=self.llm.call_with_messages(prompt)
#         res=re.search(r"【毒药决策】(.*)",result).group(1)[1:]
        res=result
        self.logger.add_log('毒药决策',self.num,self.speaker,res)
        return res
    def use_antidote(self,killed_player,strategy=''):     #解药
        cur_state=self.get_cur_state()
        prompt=cur_state+strategy+'''现在是晚上时间,%d号玩家被狼人消灭,请你根据【游戏记录】中的信息，结合【游戏策略】，请仔细思考，并是否使用解药复活他。
        请仔细思考，然后把你的决定写到【解药决策】中。
        【解药决策】:'''%(killed_player)
        result=self.llm.call_with_messages(prompt)
#         res=re.search(r"【解药决策】(.*)",result).group(1)[1:]
        res=result
        self.logger.add_log('解药决策',self.num,self.speaker,res)
        return res
    def speech(self,strategy=''):    #发言
        cur_state=self.get_cur_state()
        prompt=cur_state+strategy+'''现在是白天时间轮到你发言，请你根据【游戏记录】中的信息，结合【游戏策略】和你获得的信息，仔细思考如何发言才能帮助人类阵营获胜，并发表你的看法。
        请仔细思考，然后把发言内容写到【发言内容】中。
        【发言内容】:'''
        result=self.llm.call_with_messages(prompt)
#         res=re.search(r"【发言内容】(.*)",result).group(1)[1:]
        res=result
        self.logger.add_log('白天发言',self.num,self.speaker,res)
        return res
    def day_vote(self,strategy=''):    #白天投票
        cur_state=self.get_cur_state()
        prompt=cur_state+strategy+'''现在是白天投票时间，请你根据【游戏记录】中今天的对话和历史信息，结合【游戏策略】，仔细思考投票决定消灭哪个玩家。
        请仔细思考，然后把投票结果写到【投票结果】中。
        【投票结果】:'''
        result=self.llm.call_with_messages(prompt)
#         res=re.search(r"【投票结果】(.*)",result).group(1)[1:]
        res=result
        self.logger.add_log('白天投票',self.num,self.speaker,res)
        return res

In [316]:
#villager
class villager():
    def __init__(self,char_info,num,logger):
        self.char_info=char_info
        self.llm=base_llm(self.char_info)
        self.num=num
        self.speaker='villager'
        self.logger=logger
    def get_cur_state(self):
        return self.logger.get_cur_log(self.speaker)
    def speech(self,strategy=''):    #发言
        cur_state=self.get_cur_state()
        prompt=cur_state+strategy+'''现在是白天时间轮到你发言，请你根据【游戏记录】中的信息，结合【游戏策略】，仔细思考如何发言才能帮助人类阵营获胜，并发表你的看法。
        请仔细思考，然后把发言内容写到【发言内容】中。
        【发言内容】:'''
        result=self.llm.call_with_messages(prompt)
#         res=re.search(r"【发言内容】(.*)",result).group(1)[1:]
        res=result
        self.logger.add_log('白天发言',self.num,self.speaker,res)
        return res
    def day_vote(self,strategy=''):    #白天投票
        cur_state=self.get_cur_state()
        prompt=cur_state+strategy+'''现在是白天投票时间，请你根据【游戏记录】中今天的对话和历史信息，结合【游戏策略】，仔细思考投票决定消灭哪个玩家。
        请仔细思考，然后把投票结果写到【投票结果】中。
        【投票结果】:'''
        result=self.llm.call_with_messages(prompt)
#         res=re.search(r"【投票结果】(.*)",result).group(1)[1:]
        res=result
        self.logger.add_log('白天投票',self.num,self.speaker,res)
        return res

In [317]:
#hunter
class hunter():
    def __init__(self,char_info,num,logger):
        self.char_info=char_info
        self.llm=base_llm(self.char_info)
        self.num=num
        self.speaker='hunter'
        self.logger=logger
    def get_cur_state(self):
        return self.logger.get_cur_log(self.speaker)
    def vengence(self,strategy=''):     #复仇子弹
        cur_state=self.get_cur_state()
        prompt=cur_state+strategy+'''现在你即将被消灭，请你根据【游戏记录】中的信息，结合【游戏策略】，请仔细思考是否使用猎人的猎枪复仇消灭一名玩家，以及消灭哪位玩家。
        请仔细思考，然后把你的决定写到【猎枪决策】中。
        【猎枪决策】:'''
        result=self.llm.call_with_messages(prompt)
#         res=re.search(r"【猎枪决策】(.*)",result).group(1)[1:]
        res=result
        self.logger.add_log('猎枪决策',self.num,self.speaker,res)
        return res
    def speech(self,strategy=''):    #发言
        cur_state=self.get_cur_state()
        prompt=cur_state+strategy+'''现在是白天时间轮到你发言，请你根据【游戏记录】中的信息，结合【游戏策略】和你获得的信息，仔细思考如何发言才能帮助人类阵营获胜，并发表你的看法。
        请仔细思考，然后把发言内容写到【发言内容】中。
        【发言内容】:'''
        result=self.llm.call_with_messages(prompt)
#         res=re.search(r"【发言内容】(.*)",result).group(1)[1:]
        res=result
        self.logger.add_log('白天发言',self.num,self.speaker,res)
        return res
    def day_vote(self,strategy=''):    #白天投票
        cur_state=self.get_cur_state()
        prompt=cur_state+strategy+'''现在是白天投票时间，请你根据【游戏记录】中今天的对话和历史信息，结合【游戏策略】，仔细思考投票决定消灭哪个玩家。
        请仔细思考，然后把投票结果写到【投票结果】中。
        【投票结果】:'''
        result=self.llm.call_with_messages(prompt)
#         res=re.search(r"【投票结果】(.*)",result).group(1)[1:]
        res=result
        self.logger.add_log('白天投票',self.num,self.speaker,res)
        return res

In [319]:
#game_endcheck
def endcheck(aliv_list,char_dist):
    char_aliv=char_alive_check(aliv_list,char_dist)
    if "werewolf" not in char_aliv:
        return 2
    elif "villager" not in char_aliv:
        return 1
    elif char_aliv=={"werewolf","villager"}:
        return 1
    else:
        return 0    
endcheck(aliv_list,char_dist)

0

In [386]:
#动作信息提取
input_2_json_llm=base_llm("你是一个从对话内容提取json字符串的小助手，可以根据提示从对话中精准提取信息，并以规定的json格式返回")
def kill_vote_extr(llm,input):
    prompt='''请根据狼人的投票内容提取他的投票消灭的对象，并且以类似以下格式返回结果:{'是否消灭':'是','消灭玩家':4}
    例1：由于提供的狼人【投票结果】中并未包含具体的投票对象，而是描述了一般情况下狼人的游戏策略，因此无法直接提取出{'是否消灭':'是','消灭玩家':x}这样的信息。如果需要提取实际的投票信息，请提供包含具体投票目标的对话内容。
    【提取结果】:{'是否消灭':'否'}
    例2：狼人的【投票结果】:2号玩家。
    【提取结果】:{'是否消灭':'是','消灭玩家':2}
    #####################################
    当前狼人的【投票结果】:%s。
    【提取结果】:
    '''%(input)
    print(input)
    result=llm.call_with_messages(prompt)
    print(result)
    kill_res=eval(result)
    if kill_res['是否消灭']=='是':
        return kill_res['消灭玩家']
    else:
        return -1
def day_vote_extr(llm,input):
    prompt='''请根据当前玩家的投票内容提取他的投票消灭的对象，并且以类似以下格式返回结果:{'是否消灭':'是','消灭玩家':4}
    例1：玩家的【投票结果】:在白天的讨论中，虽然5号玩家被多次提及，我需要保持谨慎，避免过早暴露自己。0号玩家在发言中特别提到了我，这让我有些怀疑他的动机。狼人可能会试图引导村民对我产生怀疑，以转移对其他狼人成员的注意力。因此，我选择不投票给5号玩家，而是将注意力转向0号玩家，他的发言和对我行为的关注让我觉得他可能是一个狼人。当然，这只是一个策略，我需要在接下来的讨论中收集更多信息来确认我的猜测。
    【提取结果】:{'是否消灭':'是','消灭玩家':0}
    例2：玩家的【投票结果】:由于目前没有确凿的证据指向某个玩家是狼人，而且1号玩家的发言虽然试图展现新人的紧张，但也有玩家表示了他的警觉，我作为女巫需要更加谨慎。在这种情况下，我倾向于观察和收集更多的信息，避免在初期就草率投票。因此，我选择弃权，不进行投票。这样可以避免因误判而失去一位可能的队友，同时也能保持游戏的平衡，等待更多的线索出现。当然，如果其他玩家提供了有力的证据，我会重新考虑我的立场。
    【提取结果】:{'是否消灭':'否'}
    #####################################
    当前玩家的【投票结果】:%s。
    【提取结果】:
    '''%(input)
    print("决策过程:")
    print(input)
    result=llm.call_with_messages(prompt)
    print("投票文本:")
    print(result)
    kill_res=eval(result)
    if kill_res['是否消灭']=='是':
        return kill_res['消灭玩家']
    else:
        return -1
def seer_val_extr(llm,input):
    prompt='''请根据玩家的选择决策提取他的投票验证身份的对象，并且以类似以下格式返回结果:{'是否验证':'是','验证玩家':4}
    例1：玩家的【选择决策】:由于这是游戏的第1个晚上，没有之前的讨论和投票信息可供参考，所以选择验人可能会比较随机。不过，通常情况下，玩家会优先考虑验那些在开局时表现得异常活跃或者沉默不语的玩家，因为他们可能是试图掩饰自己身份的狼人。

另一种策略是验与自己关系最近的玩家，比如紧挨着你的2号玩家。这样可以初步了解你周围的安全情况。因此，作为1号玩家，我建议验2号玩家的身份。

【玩家编号】: 2
    【提取结果】:{'是否验证':'是','验证玩家':2}
    例2：玩家的【选择决策】:2号玩家。
    【提取结果】:{'是否验证':'是','验证玩家':3}
    #####################################
    当前玩家的【选择决策】:%s。
    【提取结果】:
    '''%(input)
    print(input)
    result=llm.call_with_messages(prompt)
    print(result)
    kill_res=eval(result)
    if kill_res['是否验证']=='是':
        return kill_res['验证玩家']
    else:
        return -1
def witch_poison_extr(llm,input):
    prompt='''请根据当前玩家的毒药决策提取他的决定用毒药消灭的对象，并且以类似以下格式返回结果:{'是否消灭':'是','消灭玩家':4}
    例1：玩家的【毒药决策】:根据当前的局面，我想用毒药毒死3号玩家。
    【提取结果】:{'是否消灭':'是','消灭玩家':3}
    例2：玩家的【毒药决策】:2号玩家。
    【提取结果】:{'是否消灭':'是','消灭玩家':2}
    例3:在目前的情况下，由于信息有限，我没有足够的理由直接使用毒药。因此，我的初步决定是**不使用毒药**，以待更多信息出现或出现更明确的狼人目标。然而，如果游戏中出现明显的狼人行为或有玩家的行为让我高度怀疑其为狼人，我会重新考虑这个决策。
    【提取结果】:{'是否消灭':'否'}
    #####################################
    当前玩家的【毒药决策】:%s。
    【提取结果】:
    '''%(input)
    print(input)
    result=llm.call_with_messages(prompt)
    kill_res=eval(result)
    if kill_res['是否消灭']=='是':
        return kill_res['消灭玩家']
    else:
        return -1
def witch_antidote_extr(llm,input):
    prompt='''请根据当前玩家的解药决策提取是否解救的决定，并且以类似以下格式返回结果:{'是否解救':'是'}
    例1：玩家的【解药决策】:根据当前的局面，我不想用解药救下这个人。
    【提取结果】:{'是否解救':'否'}
    例2：玩家的【解药决策】:救。
    【提取结果】:{'是否解救':'是'}
    #####################################
    当前玩家的【解药决策】:%s。
    【提取结果】:
    '''%(input)
    print(input)
    result=llm.call_with_messages(prompt)
    kill_res=eval(result)
    return kill_res['是否解救']
def hunter_vegence_extr(llm,input):
    prompt='''请根据当前玩家的猎枪决策提取他的决定用猎枪复仇消灭的对象，并且以类似以下格式返回结果:{'是否消灭':'是','消灭玩家':4}
    例1：玩家的【猎枪决策】:根据当前的局面，我想用带走3号玩家。
    【提取结果】:{'是否消灭':'是','消灭玩家':3}
    例2：玩家的【猎枪决策】:由于缺乏具体信息，我只能做出一个假设性的决定。在实际情况中，我会需要更多的游戏进程信息来进行决策。如果我是被狼人直接杀害的，而我之前没有任何信息，我的【猎枪决策】可能是不开枪，因为这样更谨慎，避免误伤好人。但如果是游戏实际进行中，这个决定应基于实际发生的事件和玩家的言行。
    【提取结果】:{'是否消灭':'否'}
    #####################################
    当前玩家的【猎枪决策】:%s。
    【提取结果】:
    '''%(input)
    print(input)
    result=llm.call_with_messages(prompt)
    kill_res=eval(result)
    if kill_res['是否消灭']=='是':
        return kill_res['消灭玩家']
    else:
        return -1

In [351]:
class game_logger():
    def __init__(self,db_name):
        self.conn=sqlite3.connect(db_name)
        self.cursor = self.conn.cursor()
        self.table_name="game_"+datetime.now().strftime("%Y%m%d%H%M%S")
        create_table_sql = f'''CREATE TABLE IF NOT EXISTS {self.table_name} (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        type TEXT NOT NULL,
        player INT NOT NULL,
        speaker TEXT NOT NULL,
        content TEXT NOT NULL
            )
        '''
        self.cursor.execute(create_table_sql)
        self.conn.commit()
    def add_log(self,log_type,num,speaker,content):
        insert_sql = f"INSERT INTO '{self.table_name}' (type,player, speaker,content) VALUES (?, ?, ?,?)"
        self.cursor.execute(insert_sql, (log_type,num, speaker,content))
        self.conn.commit()
        return 0
    def get_cur_log(self,speaker):
        if speaker=="werewolf":
            allowed_speaker={'werewolf':['夜晚狼人杀人发言','夜晚狼人杀人投票','白天发言','白天投票'],
                            'seer':['白天发言','白天投票'],
                            'witch':['白天发言','白天投票'],
                            'villager':['白天发言','白天投票'],
                            'hunter':['白天发言','白天投票'],
                            'moderater':['观察日志']}
            translator={'werewolf':'狼人',
                        'seer':'预言家',
                        'witch':'女巫',
                        'villager':'村民',
                        'hunter':'猎人',
                        'moderater':'裁判'}
            df = pd.read_sql_query(f"SELECT * FROM '{self.table_name}' ", self.conn)
            night=1
            day=0
            history_log="【游戏记录】:\n"+f"第 %d 夜:\n"%night
            for index, row in df.iterrows():
                if row['type'] in allowed_speaker[row['speaker']]:
                    if row['speaker']=='moderater':
                        if row['content']=='夜晚结束':
                            day+=1
                            history_log+=f"第 %d 天:\n"%day
                        elif row['content']=='白天结束':
                            night+=1
                            history_log+=f"第 %d 夜:\n"%night
                        else:
                            formatted_log = f"发生事件: {row['content']}"
                            history_log+=formatted_log +"\n"
                    else:
                        formatted_log = f"{row['player']}号玩家, {translator[row['speaker']]}执行 {row['type']}: {row['content']}"
                        history_log+=formatted_log +"\n"
            return history_log
        elif speaker=="seer":
            allowed_speaker={'werewolf':['白天发言','白天投票'],
                            'seer':['预言家验证身份','白天发言','白天投票'],
                            'witch':['白天发言','白天投票'],
                            'villager':['白天发言','白天投票'],
                            'hunter':['白天发言','白天投票'],
                            'moderater':['观察日志']}
            translator={'werewolf':'狼人',
                        'seer':'预言家',
                        'witch':'女巫',
                        'villager':'村民',
                        'hunter':'猎人',
                        'moderater':'裁判'}
            df = pd.read_sql_query(f"SELECT * FROM '{self.table_name}' ", self.conn)
            night=1
            day=0
            history_log="【游戏记录】:\n"+f"第 %d 夜:\n"%night
            for index, row in df.iterrows():
                if row['type'] in allowed_speaker[row['speaker']]:
                    if row['speaker']=='moderater':
                        if row['content']=='夜晚结束':
                            day+=1
                            history_log+=f"第 %d 天:\n"%day
                        elif row['content']=='白天结束':
                            night+=1
                            history_log+=f"第 %d 夜:\n"%night
                        else:
                            formatted_log = f"发生事件: {row['content']}"
                            history_log+=formatted_log +"\n"
                    else:
                        formatted_log = f"{row['player']}号玩家, {translator[row['speaker']]}执行 {row['type']}: {row['content']}"
                        history_log+=formatted_log +"\n"
            return history_log
        elif speaker=="witch":
            allowed_speaker={'werewolf':['白天发言','白天投票'],
                            'seer':['白天发言','白天投票'],
                            'witch':['解药决策','毒药决策','白天发言','白天投票'],
                            'villager':['白天发言','白天投票'],
                            'hunter':['白天发言','白天投票'],
                            'moderater':['观察日志']}
            translator={'werewolf':'狼人',
                        'seer':'预言家',
                        'witch':'女巫',
                        'villager':'村民',
                        'hunter':'猎人',
                        'moderater':'裁判'}
            df = pd.read_sql_query(f"SELECT * FROM '{self.table_name}' ", self.conn)
            night=1
            day=0
            history_log="【游戏记录】:\n"+f"第 %d 夜:\n"%night
            for index, row in df.iterrows():
                if row['type'] in allowed_speaker[row['speaker']]:
                    if row['speaker']=='moderater':
                        if row['content']=='夜晚结束':
                            day+=1
                            history_log+=f"第 %d 天:\n"%day
                        elif row['content']=='白天结束':
                            night+=1
                            history_log+=f"第 %d 夜:\n"%night
                        else:
                            formatted_log = f"发生事件: {row['content']}"
                            history_log+=formatted_log +"\n"
                    else:
                        formatted_log = f"{row['player']}号玩家, {translator[row['speaker']]}执行 {row['type']}: {row['content']}"
                        history_log+=formatted_log +"\n"
            return history_log
        else:
            allowed_speaker={'werewolf':['白天发言','白天投票'],
                            'seer':['白天发言','白天投票'],
                            'witch':['白天发言','白天投票'],
                            'villager':['白天发言','白天投票'],
                            'hunter':['白天发言','白天投票'],
                            'moderater':['观察日志']}
            translator={'werewolf':'狼人',
                        'seer':'预言家',
                        'witch':'女巫',
                        'villager':'村民',
                        'hunter':'猎人',
                        'moderater':'裁判'}
            df = pd.read_sql_query(f"SELECT * FROM '{self.table_name}' ", self.conn)
            night=1
            day=0
            history_log="【游戏记录】:\n"+f"第 %d 夜:\n"%night
            for index, row in df.iterrows():
                if row['type'] in allowed_speaker[row['speaker']]:
                    if row['speaker']=='moderater':
                        if row['content']=='夜晚结束':
                            day+=1
                            history_log+=f"第 %d 天:\n"%day
                        elif row['content']=='白天结束':
                            night+=1
                            history_log+=f"第 %d 夜:\n"%night
                        else:
                            formatted_log = f"发生事件: {row['content']}"
                            history_log+=formatted_log +"\n"
                    else:
                        formatted_log = f"{row['player']}号玩家, {translator[row['speaker']]}执行 {row['type']}: {row['content']}"
                        history_log+=formatted_log +"\n"
            return history_log
        def close_connection(self):
            self.cursor.close()
            self.conn.close()
            return 0

In [352]:
def game_init():
    logger=game_logger('werewolf.db')
    char_quota={"seer":1,"witch":1,"hunter":1,"villager":2,"werewolf":3}
    numbers=list(range(8))
    aliv_list=list(range(8))
    random.shuffle(numbers)
    char_dist={}
    char_dist[numbers[0]]="seer"
    char_dist[numbers[1]]="witch"
    char_dist[numbers[2]]="hunter"
    char_dist[numbers[3]]="villager"
    char_dist[numbers[4]]="villager"
    char_dist[numbers[5]]="werewolf"
    char_dist[numbers[6]]="werewolf"
    char_dist[numbers[7]]="werewolf"
    char_info=char_info_initialize(char_dist=char_dist)
    char_instances={}
    for i in char_dist.keys():
        if char_dist[i]=="werewolf":
            char_instances[i]=werewolf(char_info[i],i,logger)
        elif char_dist[i]=="seer":
            char_instances[i]=seer(char_info[i],i,logger)
        elif char_dist[i]=="witch":
            char_instances[i]=witch(char_info[i],i,logger)
        elif char_dist[i]=="villager":
            char_instances[i]=villager(char_info[i],i,logger)
        elif char_dist[i]=="hunter":
            char_instances[i]=hunter(char_info[i],i,logger)
    return aliv_list,char_dist,char_instances,logger

In [353]:
def night_werewolf_kill(werewolves):
    print("夜晚狼人开始行动：")
    for werewolf in werewolves:
        res=werewolf.kill_speech()
        print(f"狼人{werewolf.num}号发言:"+res)
    votes=[]
    for werewolf in werewolves:
        res = werewolf.kill_vote()
        vote_res=kill_vote_extr(input_2_json_llm,res)
        votes.append(vote_res)
    return Counter(votes).most_common(1)[0][0]
def night_witch(witch,killed_num):
    print("女巫开始行动：")
    killed_rescue=False
    poison_kill=-1
    if witch.antidote==1:
        res=witch.use_antidote(killed_num)
        print(res)
        antidote_res=witch_antidote_extr(input_2_json_llm,res)
        if antidote_res == '是':
            witch.antidote=0
            killed_rescue = True
    if witch.poison==1:
        res=witch.use_poison()
        print(res)
        poison_res=witch_poison_extr(input_2_json_llm,res)
        if poison_res != -1:
            witch.poison=0
            poison_kill=poison_res
    return killed_rescue,poison_kill         
def night_seer(seer,num,char_dist,game_logger):
    print("预言家开始行动：")
    res=seer.char_val()
    val_res=seer_val_extr(input_2_json_llm,res)
    translator={'werewolf':'狼人',
                'seer':'预言家',
                'witch':'女巫',
                'villager':'村民',
                'hunter':'猎人'}    
    content=f"{val_res}号玩家身份被验证为{translator[char_dist[val_res]]}"
    print(content)
    game_logger.add_log('预言家验证身份',num,'seer',content)
def hunter_vengence(hunter):
    vengence_kill=-1
    print("猎人即将死亡，开始复仇决策：")
    res=hunter.vengence()
    print(res)
    vengence_res=hunter_vegence_extr(input_2_json_llm,res)
    if vengence_res!= -1:
        vengence_kill=vengence_res
    return vengence_kill
def day_kill(alive_list,char_instances):
    print("白天讨论开始：")
    alive_list.sort()
    for i in alive_list:
        res=char_instances[i].speech()
        print(f"{i}:{res}")
    votes=[]
    for i in alive_list:
        res=char_instances[i].day_vote()
        vote_res=day_vote_extr(input_2_json_llm,res)
        votes.append(vote_res)
    return Counter(votes).most_common(1)[0][0]
def kill_result(alive_list,kill_list):
    for i in kill_list:
        if i in alive_list:
            alive_list.remove(i)
        else:
            print(f"{i}号玩家已经死亡，不能再次消灭！")
    return alive_list

In [372]:
aliv_list,char_dist,char_instances,logger=game_init()

In [375]:
#night process
def night(game_logger,alive_list,char_dist,char_instances):
    #初始化,确认角色分布
    aliv_char=char_alive_check(alive_list,char_dist)
    kill_list=[]
    rescue_list=[]
    #狼人杀人
    werewolves=[]
    for i in alive_list:
        if char_dist[i] == "werewolf":
            werewolves.append(char_instances[i])
    werewolf_killed=night_werewolf_kill(werewolves)
    kill_list.append(werewolf_killed)
    #女巫救人/毒人
    if 'witch' in aliv_char:
        for i in char_dist.keys():
            if char_dist[i] == 'witch':
                witch=char_instances[i]
        rescue,poison=night_witch(witch,werewolf_killed)
        kill_list.append(poison)
        rescue_list.append(rescue)
    #预言家验人
    if 'seer' in aliv_char:
        for i in char_dist.keys():
            if char_dist[i] == 'seer':
                seer=char_instances[i]
                num_seer=i
        night_seer(seer,num_seer,char_dist,game_logger)
    #夜晚结算
    killed_list_1=[x for x in kill_list if x != -1]
    rescue_list_1=[x for x in rescue_list if x != -1]
    for i in rescue_list_1:
        if i in killed_list_1:
            killed_list_1.remove(i)
    for i in killed_list_1:
        if char_dist[i] == 'hunter':
            hunter= char_instances[i]
            vengence_res=hunter_vengence(hunter)
            killed_list_1.append(vengence_res)
    killed_list_2=[x for x in killed_list_1 if x != -1]
    for i in killed_list_2:
        if i in alive_list:
            alive_list.remove(i)
    print("夜晚死亡玩家：")
    print(killed_list_2)
    print("剩余玩家：")
    print(alive_list)
    game_res= endcheck(aliv_list,char_dist)
    return game_res,alive_list

In [374]:
#day process
def day(game_logger,alive_list,char_dist,char_instances):
    #初始化
    aliv_char=char_alive_check(alive_list,char_dist)
    #发言
    day_killed=day_kill(alive_list,char_instances)
    #结算
    killed_list_1=[x for x in day_killed if x != -1]
    for i in killed_list_1:
        if char_dist[i] == 'hunter':
            hunter= char_instances[i]
            vengence_res=hunter_vengence(hunter)
            killed_list_1.append(vengence_res)
    killed_list_2=[x for x in killed_list_1 if x != -1]
    for i in killed_list_2:
        if i in alive_list:
            alive_list.remove(i)
    print("白天死亡玩家：")
    print(killed_list_2)
    print("剩余玩家：")
    print(alive_list)
    game_res= endcheck(aliv_list,char_dist)
    return game_res,alive_list

In [380]:
#main process
def game_start():
    alive_list,char_dist,char_instances,logger=game_init()
    game_result=0
    round_counter=0
    while round_counter<3:
        game_res,alive_list=night(logger,alive_list,char_dist,char_instances)
        game_result=game_res
        if game_result==1:
            print(alive_list)
            print(char_dist)
            break
        else:  
            game_res,alive_list=day(logger,alive_list,char_dist,char_instances)
            game_result=game_res
            if game_result==1:
                print(alive_list)
                print(char_dist)
                break
            else:
                round_counter+=1
                continue
    print(game_result)
    print(alive_list)
    print(char_dist)
    return 0 

In [385]:
a=game_start()
a

夜晚狼人开始行动：
狼人0号发言:【发言内容】: "各位狼队友，晚上好。作为0号玩家，我建议我们首夜击杀4号玩家，他是我们的邻居，这样可以制造混乱，让其他人难以判断狼人可能的位置。同时，4号如果是神职，他的出局会使人类阵营失去一些关键信息。如果4号是村民，他的死亡也会引起大家对周围玩家的怀疑。让我们先下手为强。"
狼人2号发言:【发言内容】: "7号，0号的提议听起来很有道理，4号确实是个威胁。不过，我们也需要考虑长远，如果4号是猎人，他被杀可能会带走一个无辜的村民，增加我们的难度。我建议考虑一下5号玩家，他与我们的距离稍远，如果他是村民，不会立即引发对我们的怀疑。如果是神职，尤其是预言家，他的出局同样能打乱人类阵营。请0号和7号分享下你们的看法。"
狼人7号发言:"2号，你的顾虑有道理。5号确实是另一个不错的选择。考虑到4号如果是猎人可能带人，而5号相对较远，更稳妥。但4号如果是预言家，他的出局对人类阵营打击更大。我觉得我们可以权衡风险，优先处理预言家。我倾向于先杀4号，如果他是预言家，我们的优势将显著增强。但如果团队觉得5号更安全，我也支持。请0号给出你的意见，我们达成一致后行动。"
【投票结果】: 4号玩家

根据三位狼人的讨论，4号玩家因为是预言家的可能性较大，且如果他是预言家，他的出局将对人类阵营造成重大打击，增强狼人优势。尽管有猎人带人风险的顾虑，但考虑到4号如果是预言家带来的影响，我认为优先击杀4号更为有利。因此，我作为0号狼人，决定投票消灭4号玩家。
{'是否消灭':'是','消灭玩家':4}
【投票结果】: 4号玩家

根据游戏记录中的讨论，三位狼人都认为4号玩家可能是预言家，且他的出局对狼人阵营更有利。虽然存在猎人带人的风险，但考虑到预言家的能力，这个风险似乎是值得冒的。因此，作为0号玩家，我会遵循之前的共识，投票消灭4号玩家。
{'是否消灭':'是','消灭玩家':4}
【投票结果】: 4号玩家
{'是否消灭':'是','消灭玩家':4}
女巫开始行动：
【解药决策】:

由于我是女巫，我的解药只能使用一次，因此必须谨慎使用。在这个情况下，我需要考虑以下几点：

1. **信息价值**：如果4号玩家是预言家，他的存活对于辨别狼人非常重要。如果他是村民，复活他的价值可能会相对较小，因为村民没有特殊能力。

2. **信任程度**：我需要根据白天的

投票文本:
```json
{
  "是否消灭": "是",
  "消灭玩家": 0
}
```


SyntaxError: invalid syntax (<string>, line 1)